<a href="https://colab.research.google.com/github/Ivywyl/jpmc-task-1/blob/main/%E2%80%9CGMVAE_Pytorch_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table align="center">
  <td align="center"><a target="_blank" href="https://github.com/jariasf/GMVAE/tree/master/pytorch">
        <img src="http://introtodeeplearning.com/images/colab/github.png"  height="70px" style="padding-bottom:5px;"  />
    <br>View Source on GitHub</a></td>
  
  <td align="center"><a target="_blank" href="https://colab.research.google.com/drive/173A4-xUYCVnc8nKCy1syKRJi7rw8B38V">
        <img src="https://www.gstatic.com/devrel-devsite/v741200ba74cbd1989790411f8b27fb588884a771dac0e0472d95190dde1f7e2f/tensorflow/images/lockup.svg"  height="70px" style="padding-bottom:5px;"  /><br>View TensorFlow Version</a></td>
  
</table>

# Gaussian Mixture Variational Autoencoder

**Author:** Jhosimar George Arias Figueroa

This notebook contains a pytorch implementation of a Gaussian Mixture Variational Autoencoder (GMVAE) applied to unsupervised clustering. The model is based on the M2 Unsupervised model proposed by Kingma et al. (https://arxiv.org/pdf/1406.5298), where instead of marginalization of the categorical variable, we use the Gumbel-Softmax distribution (https://arxiv.org/pdf/1611.01144) and modify the generative model to represent a Mixture of Gaussians.

## Load Source Code from Github

In [1]:
# clone the github repository to access source code
!git clone https://github.com/jariasf/GMVAE.git

# set the correct directory
%cd GMVAE/pytorch

fatal: destination path 'GMVAE' already exists and is not an empty directory.
/content/GMVAE/pytorch


## Install Latest Version of Pytorch

In [2]:
# tested with torch-1.3.0 torchvision-0.4.1
!pip install --upgrade torch torchvision

## Import Libraries

In [3]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
import argparse
import random
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset, DataLoader
import torch.utils.data
from scipy.io import loadmat
from model.GMVAE import *
import matplotlib.pyplot as plt
%matplotlib inline

## Input Parameters

In [4]:
#########################################################
## Input Parameters
#########################################################
parser = argparse.ArgumentParser(description='PyTorch Implementation of DGM Clustering')

## Used only in notebooks
parser.add_argument('-f', '--file',
                    help='Path for input file. First line should contain number of lines to search in')

## Dataset
parser.add_argument('--dataset', type=str,
                    default='protein', help='dataset (default: protein)')
parser.add_argument('--seed', type=int, default=1, help='random seed (default: 1)')

## GPU
parser.add_argument('--cuda', type=int, default=1,
                    help='use of cuda (default: 1)')
parser.add_argument('--gpuID', type=int, default=0,
                    help='set gpu id to use (default: 0)')

## Training
parser.add_argument('--epochs', type=int, default=100,
                    help='number of total epochs to run (default: 200)')
parser.add_argument('--batch_size', default=64, type=int,
                    help='mini-batch size (default: 64)')
parser.add_argument('--batch_size_val', default=200, type=int,
                    help='mini-batch size of validation (default: 200)')
parser.add_argument('--learning_rate', default=1e-3, type=float,
                    help='learning rate (default: 0.001)')
parser.add_argument('--decay_epoch', default=-1, type=int,
                    help='Reduces the learning rate every decay_epoch')
parser.add_argument('--lr_decay', default=0.5, type=float,
                    help='Learning rate decay for training (default: 0.5)')

## Architecture
parser.add_argument('--num_classes', type=int, default=5,
                    help='number of classes (default: 5)')
parser.add_argument('--gaussian_size', default=64, type=int,
                    help='gaussian size (default: 64)')
parser.add_argument('--input_size', default=5347, type=int,
                    help='input size (default: 5347)')

## Partition parameters
parser.add_argument('--train_proportion', default=1.0, type=float,
                    help='proportion of examples to consider for training only (default: 1.0)')

## Gumbel parameters
parser.add_argument('--init_temp', default=1.3, type=float,
                    help='Initial temperature used in gumbel-softmax (recommended 0.5-1.3, default:1.3)')
parser.add_argument('--decay_temp', default=1, type=int,
                    help='Set 1 to decay gumbel temperature at every epoch (default: 1)')
parser.add_argument('--hard_gumbel', default=0, type=int,
                    help='Set 1 to use the hard version of gumbel-softmax (default: 1)')
parser.add_argument('--min_temp', default=0.5, type=float,
                    help='Minimum temperature of gumbel-softmax after annealing (default: 0.5)' )
parser.add_argument('--decay_temp_rate', default=0.013862944, type=float,
                    help='Temperature decay rate at every epoch (default: 0.013862944)')

## Loss function parameters
parser.add_argument('--w_gauss', default=2, type=float,
                    help='weight of gaussian loss (default: 1)')
parser.add_argument('--w_categ', default=1, type=float,
                    help='weight of categorical loss (default: 1)')
parser.add_argument('--w_rec', default=1, type=float,
                    help='weight of reconstruction loss (default: 1)')
parser.add_argument('--rec_type', type=str, choices=['bce', 'mse'],
                    default='mse', help='desired reconstruction loss function (default: mse)')

## Others
parser.add_argument('--verbose', default=0, type=int,
                    help='print extra information at every epoch.(default: 0)')

args = parser.parse_args()

Set random seed in case it was specified in the parameters


In [5]:
## Random Seed
SEED = args.seed
np.random.seed(SEED)
random.seed(SEED)
torch.manual_seed(SEED)
if args.cuda:
  torch.cuda.manual_seed(SEED)

## Load Dataset

The MNIST dataset consists of 70000 handwritten digits of 28×28 pixel size and 10 classes, of which 60000 images are considered for training and 10000 images for testing. This dataset can be obtained directly from the [torchvision](https://pytorch.org/docs/stable/torchvision/datasets.html) framework.

In [6]:
print("Loading protein dataset...")

# load data
class HumanSampleDataset(Dataset):
    def __init__(self, data):
        self.data = torch.FloatTensor(data)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], 0  # 返回数据和dummy label

# 加载你的蛋白质数据
protein_df = pd.read_csv('/content/GMVAE/pytorch/protein.csv', index_col=0)  # 第一列是基因ID
print(f"原始数据形状: {protein_df.shape}")
print(f"基因数量: {protein_df.shape[0]}, 人类样本数量: {protein_df.shape[1]}")

# 转置数据：现在每行是一个人类样本，每列是一个基因的表达值
human_samples_data = protein_df.T  # shape: (n_humans, n_genes)
sample_ids = human_samples_data.index.tolist()  # 人类样本ID
gene_ids = human_samples_data.columns.tolist()  # 基因ID

print(f"聚类目标：{len(sample_ids)}个人类样本")
print(f"特征数量：{len(gene_ids)}个基因表达值")

# 数据标准化
scaler = StandardScaler()
data_normalized = scaler.fit_transform(human_samples_data.values)

# 创建数据集
train_dataset = HumanSampleDataset(data_normalized)
test_dataset = HumanSampleDataset(data_normalized)

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# print dataset shape
print('Train size: ', len(train_dataset), ' Test size: ', len(test_dataset))

Loading protein dataset...


ParserError: Error tokenizing data. C error: Expected 5775 fields in line 31, saw 7256


## Data Partition

We split the training data into train and validation according to the *train_proportion* parameter:

In [ ]:
args.batch_size = 64
args.batch_size_val = 128

def partition_dataset(n, proportion=0.8):
  train_num = int(n * proportion)
  indices = np.random.permutation(n)
  train_indices, val_indices = indices[:train_num], indices[train_num:]
  return train_indices, val_indices

if args.train_proportion == 1.0:
  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
  test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=args.batch_size_val, shuffle=False)
  val_loader = test_loader
else:
  train_indices, val_indices = partition_dataset(len(train_dataset), args.train_proportion)
  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=args.batch_size, sampler=SubsetRandomSampler(train_indices))
  val_loader = torch.utils.data.DataLoader(train_dataset, batch_size=args.batch_size_val, sampler=SubsetRandomSampler(val_indices))
  test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=args.batch_size_val, shuffle=False)

args.input_size = 5347  # 基因数量作为输入维度
args.num_classes = 5  # 人类聚类数量，可以根据需要调整
args.gaussian_size = 64  # 保持原默认值即可

print(f"输入维度: {args.input_size} (基因数量)")
print(f"聚类数量: {args.num_classes}")
print(f"样本数量: {len(sample_ids)} (人类样本)")
print(f"高斯维度: {args.gaussian_size}")

## Train Model

In [ ]:
# Model Initialization
gmvae = GMVAE(args)

In [ ]:
# Training Phase
history_loss = gmvae.train(train_loader, val_loader)

## Test Data

In [ ]:
print("Getting clustering results...")
# 1. 设置网络为评估模式
gmvae.network.eval()

# 2. 使用GMVAE自带的test方法获取基本结果
accuracy, nmi = gmvae.test(test_loader)
print(f"Accuracy: {accuracy:.5f}, NMI: {nmi:.5f}")

# 3. 如果需要详细的聚类分配，手动获取
gmvae.network.eval()
all_cluster_labels = []
all_probabilities = []

with torch.no_grad():
    for data, _ in test_loader:
        if gmvae.cuda:
            data = data.cuda()

        # 使用network进行前向传播
        out_net = gmvae.network(data, gmvae.gumbel_temp, gmvae.hard_gumbel)

        # 检查输出结构
        print("网络输出的键:", list(out_net.keys()))  # 先看看有什么输出

        # 根据输出键获取聚类概率
        if 'y_prob' in out_net:
            pi = out_net['y_prob']
        elif 'categorical' in out_net:
            pi = out_net['categorical']
        elif 'y' in out_net:
            pi = out_net['y']
        else:
            # 如果不确定，打印所有输出看看
            for key, value in out_net.items():
                print(f"{key}: shape {value.shape}")
            pi = list(out_net.values())[1]  # 通常第二个是聚类概率

        cluster_labels = torch.argmax(pi, dim=1)
        all_cluster_labels.append(cluster_labels.cpu())
        all_probabilities.append(pi.cpu())

        break  # 先只处理第一个batch看看输出结构

# 查看第一个batch的结果
print(f"第一个batch的聚类标签: {all_cluster_labels[0]}")
print(f"聚类概率形状: {all_probabilities[0].shape}")

## Complete Result

In [ ]:
# 更正后的聚类结果获取
def get_clustering_results_corrected(gmvae, test_loader, sample_ids):
    print("获取聚类结果...")
    gmvae.network.eval()

    all_cluster_labels = []
    all_probabilities = []

    with torch.no_grad():
        for data, _ in test_loader:
            if gmvae.cuda:
                data = data.cuda()

            out_net = gmvae.network(data, gmvae.gumbel_temp, gmvae.hard_gumbel)

            # 优先级顺序选择聚类概率
            if 'prob_cat' in out_net:
                pi = out_net['prob_cat']  # 最可能是聚类概率
                print("使用 prob_cat 作为聚类概率")
            elif 'categorical' in out_net:
                pi = out_net['categorical']  # 次优选择
                print("使用 categorical 作为聚类概率")
            else:
                pi = torch.softmax(out_net['logits'], dim=1)  # 从logits计算
                print("从 logits 计算聚类概率")

            cluster_labels = torch.argmax(pi, dim=1)
            all_cluster_labels.append(cluster_labels.cpu())
            all_probabilities.append(pi.cpu())

    # 合并结果
    all_cluster_labels = torch.cat(all_cluster_labels, dim=0).numpy()
    all_probabilities = torch.cat(all_probabilities, dim=0).numpy()

    return all_cluster_labels, all_probabilities

# 运行修正版本
cluster_labels, probabilities = get_clustering_results_corrected(gmvae, test_loader, sample_ids)

# 创建详细结果
cluster_results = pd.DataFrame({
    'Sample_ID': sample_ids,
    'Cluster': cluster_labels,
    'Max_Probability': np.max(probabilities, axis=1)
})

print("=== 修正后的聚类结果 ===")
for cluster_id in sorted(cluster_results['Cluster'].unique()):
    cluster_samples = cluster_results[cluster_results['Cluster'] == cluster_id]
    print(f"聚类 {cluster_id}: {len(cluster_samples)} 个样本 ({len(cluster_samples)/len(cluster_results)*100:.1f}%)")
    print(f"  平均置信度: {cluster_samples['Max_Probability'].mean():.3f}")

In [ ]:
import os
from datetime import datetime

# 1. 创建结果保存目录
results_dir = 'clustering_results'
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    print(f"创建结果目录: {results_dir}")

# 2. 生成时间戳用于文件命名
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# 3. 保存主要结果文件
main_results_file = f'{results_dir}/gmvae_clustering_results_{timestamp}.csv'
cluster_results.to_csv(main_results_file, index=False)
print(f"主要聚类结果已保存到: {main_results_file}")

# 4. 保存概率矩阵（所有聚类的概率）
prob_matrix_file = f'{results_dir}/clustering_probabilities_{timestamp}.csv'
prob_df = pd.DataFrame(probabilities,
                      columns=[f'Cluster_{i}_Prob' for i in range(probabilities.shape[1])])
prob_df['Sample_ID'] = sample_ids
prob_df = prob_df[['Sample_ID'] + [col for col in prob_df.columns if col != 'Sample_ID']]
prob_df.to_csv(prob_matrix_file, index=False)
print(f"概率矩阵已保存到: {prob_matrix_file}")

# 5. 保存聚类统计摘要
summary_file = f'{results_dir}/clustering_summary_{timestamp}.txt'
with open(summary_file, 'w') as f:
    f.write("GMVAE 聚类结果摘要\n")
    f.write("=" * 50 + "\n")
    f.write(f"总样本数: {len(cluster_results)}\n")
    f.write(f"设置聚类数: {probabilities.shape[1]}\n")
    f.write(f"实际使用聚类数: {len(cluster_results['Cluster'].unique())}\n")
    f.write(f"平均置信度: {cluster_results['Max_Probability'].mean():.4f}\n")
    f.write(f"置信度标准差: {cluster_results['Max_Probability'].std():.4f}\n\n")

    f.write("各聚类详细信息:\n")
    f.write("-" * 30 + "\n")
    for cluster_id in sorted(cluster_results['Cluster'].unique()):
        cluster_samples = cluster_results[cluster_results['Cluster'] == cluster_id]
        f.write(f"聚类 {cluster_id}:\n")
        f.write(f"  样本数量: {len(cluster_samples)} ({len(cluster_samples)/len(cluster_results)*100:.1f}%)\n")
        f.write(f"  平均置信度: {cluster_samples['Max_Probability'].mean():.4f}\n")
        f.write(f"  置信度范围: {cluster_samples['Max_Probability'].min():.4f} - {cluster_samples['Max_Probability'].max():.4f}\n")

        # 显示前5个高置信度样本
        top_samples = cluster_samples.nlargest(5, 'Max_Probability')
        f.write(f"  高置信度样本示例: {', '.join(top_samples['Sample_ID'].tolist())}\n\n")

print(f"聚类摘要已保存到: {summary_file}")

# 6. 保存每个聚类的样本列表
for cluster_id in sorted(cluster_results['Cluster'].unique()):
    cluster_samples = cluster_results[cluster_results['Cluster'] == cluster_id]
    cluster_file = f'{results_dir}/cluster_{cluster_id}_samples_{timestamp}.txt'

    with open(cluster_file, 'w') as f:
        f.write(f"聚类 {cluster_id} 样本列表\n")
        f.write(f"样本数量: {len(cluster_samples)}\n")
        f.write(f"平均置信度: {cluster_samples['Max_Probability'].mean():.4f}\n")
        f.write("=" * 40 + "\n")

        # 按置信度排序
        sorted_samples = cluster_samples.sort_values('Max_Probability', ascending=False)
        for _, sample in sorted_samples.iterrows():
            f.write(f"{sample['Sample_ID']}\t{sample['Max_Probability']:.4f}\n")

    print(f"聚类 {cluster_id} 样本列表已保存到: {cluster_file}")

# 7. 输出文件清单
print(f"\n" + "="*60)
print(f"所有结果文件已保存到目录: {results_dir}")
print(f"文件清单:")
print(f"  主要结果: {os.path.basename(main_results_file)}")
print(f"  概率矩阵: {os.path.basename(prob_matrix_file)}")
print(f"  统计摘要: {os.path.basename(summary_file)}")
for cluster_id in sorted(cluster_results['Cluster'].unique()):
    print(f"  聚类{cluster_id}样本: cluster_{cluster_id}_samples_{timestamp}.txt")
print(f"="*60)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Visualization of the feature latent space